In [33]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd




In [3]:
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(options=chrome_options)


In [37]:
#open page
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://digital.oegbverlag.at/login?r=https%3A%2F%2Fdigital.oegbverlag.at%2Fkvsystem")
time.sleep(3)

#for some reason I have to manually click on the "reject cookies" option, can't find a way to make the webdriver do it 
#select_element = driver.find_element(By.XPATH, '//*[@id="uc-center-container"]/div[2]/div/div/div/div/button[2]')
#select_element.click()


In [38]:
#select "für ögbverlag kunden"
element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button')
select_element.click()

In [39]:
#login
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")

username.send_keys("jakob.wall@econ.uzh.ch")
password.send_keys("D00rcityoverhere")

driver.find_element(By.ID, "login").click()

In [40]:
#select "erweiterte Suche and search"
element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]')
select_element.click()


In [65]:
all_years = []
for year in range(1995,1997):
    driver.get("https://digital.oegbverlag.at/kvsystem?q=&v="+str(year)+"-01-01")
    #print("01.01." + str(year))

    #select date
    #date = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/div[2]/div/input")
    #date.send_keys(Keys.CONTROL + "a")
    #date.send_keys(Keys.DELETE)
    #date.send_keys("01.01." + str(year))

    #search
    element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]/div/input")))
    searchbar = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/input")
    searchbar.send_keys(Keys.RETURN)

    #I need to click on "mehr anzeigen" until the whole list is visible. 
    #Unfortunately, the site does not work perfectly, and keeps on displaying the button "mehr anzeigen" as a clickable element even though we are at the end. As a solution, I just clik 40 times, which is always enough
    #  Unfortunately, the site does not work perf
    #select "mehr anzeigen"
    for i in range (40):
        element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button")))
        select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button')
        select_element.click()
        #need to wait a bit, otherwise the site doesn't respond
        time.sleep(2)
        print(i)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    links = soup.find_all('a')
    # Create a list to store (text, link) tuples
    result_list = []
    # Loop through each <a> tag
    for link in links:
        # Get the href attribute (the link)
        href = link.get('href')
        
        # Get the text inside the <span> (if any)
        text = link.get_text().strip()  # .strip() removes any extra whitespace
        
        if href and text:
            result_list.append((text, href))

    # Filter the list to keep only elements where the link starts with "/kvsystem"
    filtered_list = [(text, link) for text, link in result_list if link.startswith("/kvsystem/")]

    all_years.append(filtered_list)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [ ]:
#I need to click on "mehr anzeigen" until the whole list is visible. 
#Unfortunately, the site does not work perfectly, and keeps on displaying the button "mehr anzeigen" as a clickable element even though we are at the end. As a solution, I just clik 40 times, which is always enough
#  Unfortunately, the site does not work perf
#select "mehr anzeigen"
for i in range (40):
    element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button")))
    select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button')
    select_element.click()
    #need to wait a bit, otherwise the site doesn't respond
    time.sleep(2)
    print(i)

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [35]:
links = soup.find_all('a')
# Create a list to store (text, link) tuples
result_list = []
# Loop through each <a> tag
for link in links:
    # Get the href attribute (the link)
    href = link.get('href')
    
    # Get the text inside the <span> (if any)
    text = link.get_text().strip()  # .strip() removes any extra whitespace
    
    if href and text:
        result_list.append((text, href))

# Filter the list to keep only elements where the link starts with "/kvsystem"
filtered_list = [(text, link) for text, link in result_list if link.startswith("/kvsystem/")]

df = pd.DataFrame(filtered_list, columns=['Text', 'Link'])

01.01.1995
01.01.1996
01.01.1997
01.01.1998
01.01.1999
01.01.2000
01.01.2001
01.01.2002
01.01.2003
01.01.2004
01.01.2005
01.01.2006
01.01.2007
01.01.2008
01.01.2009
01.01.2010
01.01.2011
01.01.2012
01.01.2013
01.01.2014
01.01.2015
01.01.2016
01.01.2017
01.01.2018
01.01.2019
01.01.2020


4